In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [3]:
data = pd.read_csv('travel_insurance.csv')

In [4]:
data.head(5)

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commission (in value),Gender,Age
0,JZI,Airlines,Online,Value Plan,0,12,"TAIWAN, PROVINCE OF CHINA",45.0,15.75,M,39
1,EPX,Travel Agency,Online,Cancellation Plan,0,50,JAPAN,22.0,0.00,None,36
2,EPX,Travel Agency,Online,Cancellation Plan,0,251,BRAZIL,80.0,0.00,None,36
3,EPX,Travel Agency,Online,Cancellation Plan,0,6,INDIA,-10.0,0.00,None,36
4,JZI,Airlines,Online,Value Plan,0,5,CHINA,45.0,15.75,M,34


Признаки, с которыми будем работать:


Agency — название страхового агентства

Agency Type — тип страхового агентства

Distribution Channel — канал продвижения страхового агентства

Product Name — название страхового продукта

Duration — длительность поездки (количество дней)

Destination — направление поездки

Net Sales — сумма продаж ($)

Commission — комиссия страхового агентства ($)

Gender — пол застрахованного

Age — возраст застрахованного

Целевой признак:
Claim (англ. «претензия») — потребовалась ли страховая выплата: «да» — 1, «нет» — 0

In [5]:
data.describe()

,Claim,Duration,Net Sales,Commission (in value),Age
count,50660.000000,50660.000000,50660.000000,50660.000000,50660.000000
mean,0.014864,49.587150,40.749961,9.800755,39.957284
std,0.121009,103.629611,48.470030,19.716468,14.027618
min,0.000000,-2.000000,-389.000000,0.000000,0.000000
25%,0.000000,9.000000,18.000000,0.000000,35.000000
50%,0.000000,22.000000,27.000000,0.000000,36.000000
75%,0.000000,53.000000,49.000000,11.550000,43.000000
max,1.000000,4881.000000,682.000000,262.760000,118.000000



Разбейте исходные данные на две выборки:

обучающую (train);
валидационную (valid). Это 25% исходных данных.

Установите параметр (random_state) равным 12345.

Объявите четыре переменные и запишите в них:

признаки: features_train, features_valid;

целевой признак: target_train, target_valid.

Выведите на экран размеры таблиц, которые хранятся в переменных: features_train и features_valid.

In [16]:
features = data.drop(['Claim'],axis=1)

In [18]:
target = data['Claim']

In [21]:
features_train, features_valid,target_train, target_valid = train_test_split(features,target,test_size=0.25,random_state=12345)

In [23]:
print(features_train.shape)
print(features_valid.shape)

(37995, 10)
(12665, 10)


Принадлежность к категории логистическая регрессия вычисляет по формуле, состоящей из признаков. Они могут быть только численные. Наши данные содержали и категориальные признаки тоже — в этом и была ошибка.

Проверьте, признаки какого типа хранятся в таблице. Выведите их на экран. Затем напечатайте первые пять значений столбца Gender.

In [28]:
print(data.dtypes)

Agency                    object
Agency Type               object
Distribution Channel      object
Product Name              object
Claim                      int64
Duration                   int64
Destination               object
Net Sales                float64
Commission (in value)    float64
Gender                    object
Age                        int64
dtype: object


In [29]:
print(data['Gender'].head(5))

0       M
1    None
2    None
3    None
4       M
Name: Gender, dtype: object


Прямое кодирование
Преобразовать категориальные признаки в численные поможет техника прямого кодирования, или отображения (англ. One-Hot Encoding, OHE).
Принцип работы One-Hot Encoding объясним на значениях признака Gender.
Для каждого значения признака Gender (F, M, None) создаётся столбец: 
Gender_F (от англ. female, «женщина»),
Gender_M (от англ. male, «мужчина»),
Gender_None (данных о поле нет).
Когда ставить единицу в добавленные столбцы, зависит от того, каково значение Gender. Например, если F, то 1 записывается в столбец Gender_F, а M — в Gender_M. 
image

 
Резюмируем. Техникой OHE категориальные признаки переводятся в численные в два этапа:
Для каждого значения признака создаётся новый столбец;
Если объекту категория подходит, присваивается 1, если нет — 0.
Новые признаки (Gender_F, Gender_M, Gender_None) называются дамми-переменными, или дамми-признаками (англ. dummy variable, «фиктивная переменная»).
Для прямого кодирования в библиотеке pandas есть функция pd.get_dummies() (от англ. «получить фиктивные переменные»).

## Преобразуйте колонку Gender техникой OHE.
Вызовите функцию pd.get_dummies() и напечатайте на экране первые пять записей изменённой таблицы.

In [32]:
print(pd.get_dummies(data['Gender'].head(5)))

   M  None
0  1     0
1  0     1
2  0     1
3  0     1
4  1     0


Когда данных в избытке, можно угодить в ловушку фиктивных признаков.

### Дамми-ловушка

Пример:
тобы подать документы на шенгенскую визу, нужно доказать, что деньги у вас есть.

Вы решили перестраховаться, поэтому взяли и выписку с банковского счёта, и справку с работы, и 2-НДФЛ. Хотя визовому центру достаточно двух документов. Вашей модели лишняя информация тоже не очень-то нужна. 

Если оставить всё как есть, обучаться она будет сложнее. 
В таблицу добавились три новых столбца. Поскольку они сильно связаны между собой, один удалим без сожаления. Восстановить столбец можно по оставшимся двум. Так мы не попадём в дамми-ловушку (англ. dummy trap, «ловушка фиктивных признаков»).


Столбец уберём вызовом функции:

### pd.get_dummies() с аргументом drop_first (от англ. «сбросьте первый»).

Он удаляет первую колонку и передаётся как drop_first=True или drop_first=False (True — первый столбец сбрасывается, False — не сбрасывается).

In [35]:
print(pd.get_dummies(data['Gender'],drop_first=True).head(5))

   M  None
0  1     0
1  0     1
2  0     1
3  0     1
4  1     0



Примените прямое кодирование ко всему датафрейму. Вызовите функцию pd.get_dummies() c аргументом drop_first. Сохраните таблицу в переменной data_ohe.
Выведите на экран первые три строки преобразованной таблицы.

In [37]:
data_ohe = pd.get_dummies(data,drop_first=True)

In [39]:
data_ohe.head(3)

,Claim,Duration,Net Sales,Commission (in value),Age,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,...,Destination_UNITED STATES,Destination_URUGUAY,Destination_UZBEKISTAN,Destination_VANUATU,Destination_VENEZUELA,Destination_VIET NAM,Destination_ZAMBIA,Destination_ZIMBABWE,Gender_M,Gender_None
0,0,12,45.0,15.75,39,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,50,22.0,0.00,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,251,80.0,0.00,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разбейте исходные данные на две выборки в соотношении 75:25 (%):
обучающую (train);
валидационную (valid).

Объявите четыре переменные и запишите в них:
признаки: features_train, features_valid;
целевой признак: target_train, target_valid.

Вам предстоит освоить альтернативный способ работы с функцией train_test_split(): когда на вход подаются две переменные (признаки и целевой признак). Поработайте с документацией.

Обучите логистическую регрессию.
Напечатайте на экране текст "Обучено!" (уже в прекоде). Так вы убедитесь, что код выполнился без ошибок.

Вложите и в train_test_split(), и в LogisticRegression() параметр random_state, равный 12345.

In [57]:
data_ohe = pd.get_dummies(data, drop_first=True)
target = data_ohe['Claim']
features = data_ohe.drop('Claim', axis=1)

In [60]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 12345)

In [61]:
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_train, target_train)
print("Обучено!")

Обучено!
